In [9]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim

import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [30]:
trainset.train_data.shape

(50000, 32, 32, 3)

In [29]:
class rand_Net3(nn.Module):
    def __init__(self):
        super(rand_Net3, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3,1)
        self.conv2 = nn.Conv2d(32, 64, 3,1 )
        self.conv3 = nn.Conv2d(64, 256, 3,1 )
        self.fc = nn.Linear(1024, 10)
        self.conv1.train(False)
        self.conv2.train(False)
        self.conv3.train(False)

        #for k in range(self.conv1.weight.data.shape[1]):
        #    self.conv1.weight.data[k,0,:,:] = torch.FloatTensor(np.matmul(np.random.random(-1.,1.,(5,1)),np.random.random(-1.,1.,(1,5))))
        #for k in range(self.conv2.weight.data.shape[1]):
        #    self.conv2.weight.data[k,0,:,:] = torch.FloatTensor(np.matmul(np.random.random(-1.,1.,(5,1)),np.random.random(-1.,1.,(1,5))))
        #for k in range(self.conv3.weight.data.shape[1]):
        #    self.conv3.weight.data[k,0,:,:] = torch.FloatTensor(np.matmul(np.random.random((5,1)),np.random.random((1,5))))

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x),kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv3(x),kernel_size=2, stride=2))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x, x
    
class rand_Net(nn.Module):
    def __init__(self):
        super(rand_Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        #x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(x.size(0), -1)
        return x

In [34]:
net = rand_Net3()
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients

        # forward + backward + optimize
        outputs, fcw = net(inputs)
        l1 = torch.abs(fcw).sum()
        
        criterion = nn.CrossEntropyLoss() 
        optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
        optimizer.zero_grad()
        
        #print('CRIT', criterion(outputs, labels))
        #print('L1', l1)
        
        loss = criterion(outputs, labels) + 0.005*l1
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.301
[1,  4000] loss: 2.289
[1,  6000] loss: 2.270
[2,  2000] loss: 2.237
[2,  4000] loss: 2.206
[2,  6000] loss: 2.176
[3,  2000] loss: 2.143
[3,  4000] loss: 2.118
[3,  6000] loss: 2.080
[4,  2000] loss: 2.048
[4,  4000] loss: 2.010
[4,  6000] loss: 1.985
[5,  2000] loss: 1.947
[5,  4000] loss: 1.934
[5,  6000] loss: 1.916
[6,  2000] loss: 1.882
[6,  4000] loss: 1.879
[6,  6000] loss: 1.860
[7,  2000] loss: 1.834
[7,  4000] loss: 1.827
[7,  6000] loss: 1.805
[8,  2000] loss: 1.798
[8,  4000] loss: 1.767
[8,  6000] loss: 1.768
[9,  2000] loss: 1.744
[9,  4000] loss: 1.739
[9,  6000] loss: 1.724
[10,  2000] loss: 1.709
[10,  4000] loss: 1.687
[10,  6000] loss: 1.694
[11,  2000] loss: 1.670
[11,  4000] loss: 1.664
[11,  6000] loss: 1.652
[12,  2000] loss: 1.641
[12,  4000] loss: 1.631
[12,  6000] loss: 1.615
[13,  2000] loss: 1.603
[13,  4000] loss: 1.606
[13,  6000] loss: 1.588
[14,  2000] loss: 1.574
[14,  4000] loss: 1.572
[14,  6000] loss: 1.561
[15,  2000] loss: 1

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs,_ = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(8):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [21]:
from sklearn import linear_model,metrics,preprocessing
from sklearn.ensemble import RandomForestClassifier

print('compute purely on pixels')
for ty,data in enumerate(trainloader):
    images, labels = data
    for alpha in [1]:#np.logspace(0,3,10):
        print alpha
        reg = linear_model.LogisticRegression(penalty='l1',solver ='saga',C = alpha,n_jobs=6,max_iter=3)
        rf = RandomForestClassifier(n_estimators = 500,n_jobs=6)
        reg.fit(images[:,:,:,:].numpy().reshape(7000,-1) , labels) 
        rf.fit(images[:,:,:,:].numpy().reshape(7000,-1) , labels) 
    for mod in [reg,rf]:
        print ("-----")
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        for a,data in enumerate(testloader):
            imagesp, labelsp = data
            predicted = mod.predict(imagesp[:,:,:,:].numpy().reshape(500,-1))
            c = (predicted == labelsp).squeeze()
            for i in range(500):
                label = labelsp[i]
                class_correct[label] += c[i]
                class_total[label] += 1
            if a>=4:
                break 
        for i in range(10):
            print('Accuracy of %5s : %2d %%' % (
                classes[i], 100 * class_correct[i] / class_total[i]))
        print np.sum(class_correct)/np.sum(class_total)
    break



compute purely on pixels
1
-----
Accuracy of plane : 44 %
Accuracy of   car : 46 %
Accuracy of  bird : 21 %
Accuracy of   cat : 19 %
Accuracy of  deer : 30 %
Accuracy of   dog : 30 %
Accuracy of  frog : 53 %
Accuracy of horse : 45 %
Accuracy of  ship : 50 %
Accuracy of truck : 38 %
0.3828
-----
Accuracy of plane : 44 %
Accuracy of   car : 47 %
Accuracy of  bird : 29 %
Accuracy of   cat : 16 %
Accuracy of  deer : 34 %
Accuracy of   dog : 40 %
Accuracy of  frog : 59 %
Accuracy of horse : 45 %
Accuracy of  ship : 64 %
Accuracy of truck : 56 %
0.4408


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model,metrics,preprocessing


for ty,data in enumerate(trainloader):
    mrand_Net = rand_Net_3() 
    images, labels = data
    outputs = mrand_Net(Variable(images))
    if True: #for nest in [500]:#np.logspace(0,3,10):
        print('nest is %s'%nest)
        print(outputs.data.numpy().shape, labels.shape)
        rf = RandomForestClassifier(n_estimators=700, n_jobs=6)
        reg = linear_model.LogisticRegression(penalty='l1',solver ='saga',C = alpha,n_jobs=6,max_iter=300)
        
        rf.fit(outputs.data.numpy(), labels) 
        reg.fit(outputs.data.numpy(), labels) 
        
        for mod in [rf,reg]:
            print(mod)
            class_correct = list(0. for i in range(10))
            class_total = list(0. for i in range(10))
            for a, data in enumerate(testloader):
                imagesp, labelsp = data
                outputsp = mrand_Net(Variable(imagesp))
                predicted = mod.predict(outputsp.data.numpy())
                c = (predicted == labelsp).squeeze()
                for i in range(500):
                    label = labelsp[i]
                    class_correct[label] += c[i]
                    class_total[label] += 1
                if a>=4:
                    break   
            for i in range(10):
                print('Accuracy of %5s : %2d %%' % (
                    classes[i], 100 * class_correct[i] / class_total[i]))
            print np.sum(class_correct)/np.sum(class_total)
        #L.append(metrics.log_loss(np.array(labels_t),yout ,labels=range(10)))       
    #print(L, L_lr) 
    break

nest is 500
((7000, 512), torch.Size([7000]))
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=700, n_jobs=6,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy of plane : 40 %
Accuracy of   car : 46 %
Accuracy of  bird : 21 %
Accuracy of   cat :  9 %
Accuracy of  deer : 29 %
Accuracy of   dog : 37 %
Accuracy of  frog : 56 %
Accuracy of horse : 44 %
Accuracy of  ship : 51 %
Accuracy of truck : 39 %
0.3788
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=300, multi_class='ovr', n_jobs=6,
          penalty='l1', random_state=None, solver='saga', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy of plane